In [47]:
import pandas as pd
import sklearn 
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np
import gc

In [43]:
file_1 = pd.read_csv("open_data_train_valid/train/train_1.txt",chunksize=100,sep='\t')
file_1 = file_1.get_chunk()
columns = file_1.columns
data_type = file_1.dtypes.copy()
id_1 = data_type[data_type== 'int64'].index
id_2 = data_type[data_type== 'float64'].index
d = {}
for each in id_1:
    d[each] = np.int32
for each in id_2:
    d[each] = np.float32

In [44]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep = True).sum()
    else:
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b/1024 **2
    return '{:03.2f}MB'.format(usage_mb)

In [55]:
names = []
for i in range(2,6):
    names.append('open_data_train_valid/train/train_%d.txt' %i)
labelled_iter = pd.read_csv("open_data_train_valid/train/train_1.txt",chunksize=20000,sep='\t',dtype =d)
total = labelled_iter.get_chunk()
for name in names:
    unlabelled_iter = pd.read_csv(name,chunksize=20000,sep='\t',dtype =d,names = columns)
    unlabelled = unlabelled_iter.get_chunk()
    total = pd.concat([total,unlabelled],axis=0)
    del unlabelled
    gc.collect()

In [57]:
mem_usage(total)

'2581.23MB'

In [70]:
unlabeled_index = total['label'][total['label'].isnull() == True].index
labeled_index = total['label'][total['label'].isnull() == False].index

In [74]:
tag = total['tag']
label = total['label']

In [76]:
total.drop(['loan_dt','id','label','tag'],axis = 1,inplace = True)

In [59]:
'''
total.index = total['id']
indices_labelled = labelled['id']
indices_unlabelled = unlabelled['id']
tag = total['tag']
label = labelled['label']
label.index = labelled['id']
total.drop(['loan_dt','id','label','tag'],axis = 1,inplace = True)
'''

NameError: name 'unlabelled' is not defined

In [7]:
'''
file_5 = "open_data_train_valid/train/train_5.txt"
file_1 = "open_data_train_valid/train/train_1.txt"

unlabelled_iter = pd.read_csv('open_data_train_valid/train/train_5.txt',chunksize=10000,sep='\t',dtype =d)

unlabelled = unlabelled_iter.get_chunk()
unlabelled.columns = labelled.columns
total = pd.concat([labelled,unlabelled],axis=0)
total.index = total['id']
indices_labelled = labelled['id']
indices_unlabelled = unlabelled['id']
tag = total['tag']
label = labelled['label']
label.index = labelled['id']
total.drop(['loan_dt','id','label','tag'],axis = 1,inplace = True)
'''

In [36]:
'''
labelled_iter = pd.read_csv("open_data_train_valid/train/train_1.txt",chunksize=20000,sep='\t',dtype =d)
labelled = labelled_iter.get_chunk()
columns = labelled.columns
unlabelled_iter = pd.read_csv('open_data_train_valid/train/train_5.txt',chunksize=10000,sep='\t',dtype =d,header=None,
                              names = columns)
unlabelled = unlabelled_iter.get_chunk()
'''

In [78]:
gc.collect()

187

In [80]:
for column in total.columns:
    if np.sum(np.isnan(total[column])) == total[column].shape[0]:
        total.drop(column,axis=1,inplace = True)

In [81]:
Q3 = total.quantile(q = 0.75,axis = 0)
Q1 = total.quantile(q = 0.25,axis = 0)
Q_up = Q3+1.5*(Q3-Q1)
Q_low = Q3-1.5*(Q3-Q1)
scaled = total.ge(Q_low.T,axis=1)&total.le(Q_up.T,axis=1)
total = total[scaled]

In [79]:
#丢弃分布过于极端的数据（某一值占比超过80%)
#for column in total.columns:
#    if ((total[column].value_counts() / total[column].value_counts().sum()) > 0.8).any() == True:
#        total.drop(column, 1,inplace = True)
#处理缺失值大于80%
#区分属性、数值数据
for column in total.columns:
    if np.sum(np.isnan(total[column])) == total[column].shape[0]:
        total.drop(column,axis=1,inplace = True)
Q3 = total.quantile(q = 0.75,axis = 0)
Q1 = total.quantile(q = 0.25,axis = 0)
Q_up = Q3+1.5*(Q3-Q1)
Q_low = Q3-1.5*(Q3-Q1)
scaled = total.ge(Q_low.T,axis=1)&total.le(Q_up.T,axis=1)
total = total[scaled]
indices = total.dtypes[total.dtypes == 'int64'].index
for index in indices:
    if len(total.ix[:,index].unique())>300:
        indices = indices.drop(index)
categorical_feature = total.ix[:,indices]
numerical_feature = total.ix[:,total.columns.drop(indices)]
#填补缺失值
numerical_feature.fillna(numerical_feature.mean(axis=0,numeric_only =float),inplace = True)
categorical_feature.fillna(-5,inplace=True)
#归一化
numerical_feature = numerical_feature.apply(lambda x: (x-x.mean())/x.std(),axis = 1)
categorical_feature = categorical_feature - categorical_feature.mean()

total_scaled = pd.concat([numerical_feature,categorical_feature],axis=1)


KeyboardInterrupt: 

In [ ]:
#Kmeans
y_pred = KMeans(n_clusters = 6,max_iter= ).fit_predict(total_scaled)
total_scaled['group'] = y_pred
group_1 = total_scaled[total_scaled['group']==0]
group_1_labelled = group_1.loc[group_1.index.isin(indices_labelled)]
group_1_unlabelled = group_1.loc[group_1.index.isin(indices_unlabelled)]
group_2 = total_scaled[total_scaled['group']==1]
group_2_labelled = group_2.loc[group_2.index.isin(indices_labelled)]
group_2_unlabelled = group_2.loc[group_2.index.isin(indices_unlabelled)]

In [179]:
y_pred = KMeans(n_clusters = 4,max_iter= 1000).fit_predict(total_scaled)

In [175]:
y = pd.Series(y_pred).value_counts()

In [180]:
y

3    9569
1    6057
0    3590
2     784
dtype: int64